# Covid-19 Report

In this project we are going through the covid-19 data from the [John Hopkins University](https://github.com/CSSEGISandData/COVID-19) to build a full world status report. This project is divided in 3 parts:

* Setting up the data
* Exploratory data analysis
* Build the report

## Setting up the data

We start our project by loading the need packages and the John Hopkins University data.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from fpdf import FPDF
import plotly.express as px
from datetime import datetime, timedelta
import pycountry

confirmed_link = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
death_link = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
recovered_link = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'

confirmed_df = pd.read_csv(confirmed_link)
confirmed_df.tail()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
284,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,703228,703228,703228,703228,703228,703228,703228,703228,703228,703228
285,NaN,Winter Olympics 2022,39.904200,116.407400,0,0,0,0,0,0,...,535,535,535,535,535,535,535,535,535,535
286,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,11945,11945,11945,11945,11945,11945,11945,11945,11945,11945
287,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,343012,343012,343079,343079,343079,343135,343135,343135,343135,343135
288,NaN,Zimbabwe,-19.015438,29.154857,0,0,0,0,0,0,...,263921,264127,264127,264127,264127,264127,264127,264127,264276,264276


In [2]:
death_df = pd.read_csv(death_link)
death_df.tail()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
284,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,5708,5708,5708,5708,5708,5708,5708,5708,5708,5708
285,NaN,Winter Olympics 2022,39.904200,116.407400,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
286,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,2159,2159,2159,2159,2159,2159,2159,2159,2159,2159
287,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,4057,4057,4057,4057,4057,4057,4057,4057,4057,4057
288,NaN,Zimbabwe,-19.015438,29.154857,0,0,0,0,0,0,...,5663,5668,5668,5668,5668,5668,5668,5668,5671,5671


In [3]:
recovered_df = pd.read_csv(recovered_link)
recovered_df.tail()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
269,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
270,NaN,Winter Olympics 2022,39.904200,116.407400,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
271,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
272,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
273,NaN,Zimbabwe,-19.015438,29.154857,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We have 3 different dataframes containing information about the confirmed, the death and the recovered cases. The data format is such that there is a row per country and a column per date. There are 299 rows in the confirmed and death dataframes and 274 in the recovered dataframe, which means that we don't have the information for the recovered cases for all countries.

Let us reshape the data into a more suitable format, so that we can have a column with the dates and a column with the cases.

In [4]:
confirmed_df = pd.melt(confirmed_df, id_vars=confirmed_df.columns[0:4], value_vars=confirmed_df.columns[4:],
        var_name='Date', value_name='Cases')
confirmed_df.tail()

,Province/State,Country/Region,Lat,Long,Date,Cases
330322,NaN,West Bank and Gaza,31.952200,35.233200,3/9/23,703228
330323,NaN,Winter Olympics 2022,39.904200,116.407400,3/9/23,535
330324,NaN,Yemen,15.552727,48.516388,3/9/23,11945
330325,NaN,Zambia,-13.133897,27.849332,3/9/23,343135
330326,NaN,Zimbabwe,-19.015438,29.154857,3/9/23,264276


In [5]:
death_df = pd.melt(death_df, id_vars=death_df.columns[0:4], value_vars=death_df.columns[4:],
        var_name='Date', value_name='Cases')
death_df.tail()

,Province/State,Country/Region,Lat,Long,Date,Cases
330322,NaN,West Bank and Gaza,31.952200,35.233200,3/9/23,5708
330323,NaN,Winter Olympics 2022,39.904200,116.407400,3/9/23,0
330324,NaN,Yemen,15.552727,48.516388,3/9/23,2159
330325,NaN,Zambia,-13.133897,27.849332,3/9/23,4057
330326,NaN,Zimbabwe,-19.015438,29.154857,3/9/23,5671


In [6]:
recovered_df = pd.melt(recovered_df, id_vars=recovered_df.columns[0:4], value_vars=recovered_df.columns[4:],
        var_name='Date', value_name='Cases')
recovered_df.tail()

,Province/State,Country/Region,Lat,Long,Date,Cases
313177,NaN,West Bank and Gaza,31.952200,35.233200,3/9/23,0
313178,NaN,Winter Olympics 2022,39.904200,116.407400,3/9/23,0
313179,NaN,Yemen,15.552727,48.516388,3/9/23,0
313180,NaN,Zambia,-13.133897,27.849332,3/9/23,0
313181,NaN,Zimbabwe,-19.015438,29.154857,3/9/23,0


Now let us check for any missing values.

In [7]:
confirmed_df.isna().sum()

Province/State    226314
Country/Region         0
Lat                 2286
Long                2286
Date                   0
Cases                  0
dtype: int64

In [8]:
death_df.isna().sum()

Province/State    226314
Country/Region         0
Lat                 2286
Long                2286
Date                   0
Cases                  0
dtype: int64

In [9]:
recovered_df.isna().sum()

Province/State    227457
Country/Region         0
Lat                 1143
Long                1143
Date                   0
Cases                  0
dtype: int64

Most of the Province/State values are missing, therefore we are going to remove this column and group the Cases by the Country/Region column. We are also removing the Lat and Long rows, since we will be using the pycountry package to plot some maps visualizations.

In [10]:
confirmed_df = confirmed_df.iloc[:, [1, 4, 5]]
death_df = death_df.iloc[:, [1, 4, 5]]
recovered_df = recovered_df.iloc[:, [1, 4, 5]]
confirmed_df['Cases'] = confirmed_df.groupby(['Country/Region', 'Date'])['Cases'].transform('sum')
confirmed_df = confirmed_df.drop_duplicates(subset=['Country/Region', 'Date', 'Cases']).reset_index(drop=True)
confirmed_df.tail()

,Country/Region,Date,Cases
229738,West Bank and Gaza,3/9/23,703228
229739,Winter Olympics 2022,3/9/23,535
229740,Yemen,3/9/23,11945
229741,Zambia,3/9/23,343135
229742,Zimbabwe,3/9/23,264276


In [11]:
death_df['Cases'] = death_df.groupby(['Country/Region', 'Date'])['Cases'].transform('sum')
death_df = death_df.drop_duplicates(subset=['Country/Region', 'Date', 'Cases']).reset_index(drop=True)
death_df.tail()

,Country/Region,Date,Cases
229738,West Bank and Gaza,3/9/23,5708
229739,Winter Olympics 2022,3/9/23,0
229740,Yemen,3/9/23,2159
229741,Zambia,3/9/23,4057
229742,Zimbabwe,3/9/23,5671


In [12]:
recovered_df['Cases'] = recovered_df.groupby(['Country/Region', 'Date'])['Cases'].transform('sum')
recovered_df = recovered_df.drop_duplicates(subset=['Country/Region', 'Date', 'Cases']).reset_index(drop=True)
recovered_df.tail()

,Country/Region,Date,Cases
229738,West Bank and Gaza,3/9/23,0
229739,Winter Olympics 2022,3/9/23,0
229740,Yemen,3/9/23,0
229741,Zambia,3/9/23,0
229742,Zimbabwe,3/9/23,0


Now all the dataframes have the same number of rows (229743). For each dataframe, the column Cases represents the accumulated number of cases for a specific country. Let us create a column for the daily new cases. For doing so, we will need to order the rows by the Date field. Therefore we have to change its type to datetime.

In [13]:
confirmed_df['Date'] = pd.to_datetime(confirmed_df['Date'])
death_df['Date'] = pd.to_datetime(death_df['Date'])
recovered_df['Date'] = pd.to_datetime(recovered_df['Date'])
confirmed_df = confirmed_df.sort_values(['Country/Region', 'Date']).reset_index(drop=True)
death_df = death_df.sort_values(['Country/Region', 'Date']).reset_index(drop=True)
recovered_df = recovered_df.sort_values(['Country/Region', 'Date']).reset_index(drop=True)

confirmed_df['New_Cases'] = confirmed_df.groupby('Country/Region')['Cases'].transform('diff')
confirmed_df.head()

,Country/Region,Date,Cases,New_Cases
0,Afghanistan,2020-01-22,0,NaN
1,Afghanistan,2020-01-23,0,0.0
2,Afghanistan,2020-01-24,0,0.0
3,Afghanistan,2020-01-25,0,0.0
4,Afghanistan,2020-01-26,0,0.0


In [14]:
death_df['New_Cases'] = death_df.groupby('Country/Region')['Cases'].transform('diff')
death_df.head()

,Country/Region,Date,Cases,New_Cases
0,Afghanistan,2020-01-22,0,NaN
1,Afghanistan,2020-01-23,0,0.0
2,Afghanistan,2020-01-24,0,0.0
3,Afghanistan,2020-01-25,0,0.0
4,Afghanistan,2020-01-26,0,0.0


In [15]:
recovered_df['New_Cases'] = recovered_df.groupby('Country/Region')['Cases'].transform('diff')
recovered_df.head()

,Country/Region,Date,Cases,New_Cases
0,Afghanistan,2020-01-22,0,NaN
1,Afghanistan,2020-01-23,0,0.0
2,Afghanistan,2020-01-24,0,0.0
3,Afghanistan,2020-01-25,0,0.0
4,Afghanistan,2020-01-26,0,0.0


Now, for each dataframe, the column New_Cases will have a null value for every single country. In order to correct this, we have to replace the null values with its correspondent value in the Cases column.

In [16]:
confirmed_df['New_Cases'] = np.where(confirmed_df['New_Cases'].isna(), confirmed_df['Cases'], confirmed_df['New_Cases'])
confirmed_df.head()

,Country/Region,Date,Cases,New_Cases
0,Afghanistan,2020-01-22,0,0.0
1,Afghanistan,2020-01-23,0,0.0
2,Afghanistan,2020-01-24,0,0.0
3,Afghanistan,2020-01-25,0,0.0
4,Afghanistan,2020-01-26,0,0.0


In [17]:
death_df['New_Cases'] = np.where(death_df['New_Cases'].isna(), death_df['Cases'], death_df['New_Cases'])
death_df.head()

,Country/Region,Date,Cases,New_Cases
0,Afghanistan,2020-01-22,0,0.0
1,Afghanistan,2020-01-23,0,0.0
2,Afghanistan,2020-01-24,0,0.0
3,Afghanistan,2020-01-25,0,0.0
4,Afghanistan,2020-01-26,0,0.0


In [18]:
recovered_df['New_Cases'] = np.where(recovered_df['New_Cases'].isna(), recovered_df['Cases'], recovered_df['New_Cases'])
recovered_df.head()

,Country/Region,Date,Cases,New_Cases
0,Afghanistan,2020-01-22,0,0.0
1,Afghanistan,2020-01-23,0,0.0
2,Afghanistan,2020-01-24,0,0.0
3,Afghanistan,2020-01-25,0,0.0
4,Afghanistan,2020-01-26,0,0.0


To finish our data cleaning process, we are going to merge the three dataframes, change the Country/Region column name to Country, get the country codes, remove the non country cases and save the data in this tidy format in a csv file.

In [19]:
covid_df = confirmed_df.merge(death_df, on=['Country/Region', 'Date'], suffixes=['_Confirmed', '_Death'])
covid_df = covid_df.merge(recovered_df, on=['Country/Region', 'Date'])
covid_df = covid_df.rename(columns={'Country/Region':'Country', 'Cases':'Cases_Recovered', 'New_Cases':'New_Cases_Recovered'})
covid_df.tail()

,Country,Date,Cases_Confirmed,New_Cases_Confirmed,Cases_Death,New_Cases_Death,Cases_Recovered,New_Cases_Recovered
229738,Zimbabwe,2023-03-05,264127,0.0,5668,0.0,0,0.0
229739,Zimbabwe,2023-03-06,264127,0.0,5668,0.0,0,0.0
229740,Zimbabwe,2023-03-07,264127,0.0,5668,0.0,0,0.0
229741,Zimbabwe,2023-03-08,264276,149.0,5671,3.0,0,0.0
229742,Zimbabwe,2023-03-09,264276,0.0,5671,0.0,0,0.0


In [26]:
code_map = {'Brunei':'BRN',
           'Burma':'MMR',
           'Congo (Brazzaville)':'COG',
           'Congo (Kinshasa)':'COD',
           "Cote d'Ivoire":'CIV',
           'Holy See':'VAT',
           'Iran':'IRN',
           'Korea, North':'PRK',
           'Korea, South':'KOR',
           'Kosovo':'XXK',
           'Laos':'LAO',
           'Micronesia':'FSM',
           'Russia': 'RUS',
           'Syria':'SYR',
           'Taiwan*':'TWN',
           'West Bank and Gaza':'PSE'}
def get_country_code(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except:
        try:
            return code_map[name]
        except:
            None
covid_df['Code'] = covid_df['Country'].apply(get_country_code)
covid_df.tail()

,Country,Date,Cases_Confirmed,New_Cases_Confirmed,Cases_Death,New_Cases_Death,Cases_Recovered,New_Cases_Recovered,Code
229738,Zimbabwe,2023-03-05,264127,0.0,5668,0.0,0,0.0,ZWE
229739,Zimbabwe,2023-03-06,264127,0.0,5668,0.0,0,0.0,ZWE
229740,Zimbabwe,2023-03-07,264127,0.0,5668,0.0,0,0.0,ZWE
229741,Zimbabwe,2023-03-08,264276,149.0,5671,3.0,0,0.0,ZWE
229742,Zimbabwe,2023-03-09,264276,0.0,5671,0.0,0,0.0,ZWE


In [29]:
covid_df = covid_df[~covid_df['Code'].isna()].reset_index(drop=True)
covid_df.to_csv('data/covid_tidy_data.csv', index=False, float_format='%.0f')
covid_df.tail()

,Country,Date,Cases_Confirmed,New_Cases_Confirmed,Cases_Death,New_Cases_Death,Cases_Recovered,New_Cases_Recovered,Code
225166,Zimbabwe,2023-03-05,264127,0.0,5668,0.0,0,0.0,ZWE
225167,Zimbabwe,2023-03-06,264127,0.0,5668,0.0,0,0.0,ZWE
225168,Zimbabwe,2023-03-07,264127,0.0,5668,0.0,0,0.0,ZWE
225169,Zimbabwe,2023-03-08,264276,149.0,5671,3.0,0,0.0,ZWE
225170,Zimbabwe,2023-03-09,264276,0.0,5671,0.0,0,0.0,ZWE


We ended up with 225171 ro